<div style="font-size: 200%; font-weight: bold; color: maroon;">GraphX</div>
<!-- Paulo Villegas, 2021 - v. 2.0 -->

Hacemos algunas operaciones sencillas usando la API de [GraphX](https://spark.apache.org/docs/latest/graphx-programming-guide.html). Ejemplo tomado de [ampcamp Berkeley](http://ampcamp.berkeley.edu/big-data-mini-course/graph-analytics-with-graphx.html)

<div class="alert alert-warning"> Nota: este notebook supone que estamos usando el <a href="http://almond.sh">Almond Scala kernel</a> para Jupyter, disponible en la máquina virtual con Spark 3</div>

In [ ]:
scala.util.Properties.versionNumberString

In [ ]:
// Load the Spark Jars into the Scala interpreter in the Almond kernel
def loadSpark(path: Option[String] = None ) = {
    val jardir = path.getOrElse("/opt/spark/current") + "/jars"
    val jars = java.nio.file.Files.list(java.nio.file.Paths.get(jardir)).toArray.map(_.toString);
    val seq = jars.map( fname => ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(fname)) );
    interp.load.cp(seq);
}

loadSpark(sys.env.get("SPARK_HOME"));

In [ ]:
// Lower log level to WARN
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org.apache").setLevel(Level.WARN)

In [ ]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

val conf = new SparkConf().setAppName("GraphX").setMaster("local[*]")
val sc = SparkContext.getOrCreate(conf)

# Preparación de datos

In [ ]:
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx._

<img src='img/social_graph.png' width="500"/>

In [ ]:
// Define an array de vertices
val vertexArray = Array(
  (1L, ("Alice", 28)),
  (2L, ("Bob", 27)),
  (3L, ("Charlie", 65)),
  (4L, ("David", 42)),
  (5L, ("Ed", 55)),
  (6L, ("Fran", 50))
  )

In [ ]:
// Define an array de edges between the defined vertices
val edgeArray = Array(
  Edge(2L, 1L, 7),
  Edge(2L, 4L, 2),
  Edge(3L, 2L, 4),
  Edge(3L, 6L, 3),
  Edge(4L, 1L, 1),
  Edge(5L, 2L, 2),
  Edge(5L, 3L, 8),
  Edge(5L, 6L, 3)
  )

# Creación del grafo

Prepara los RDDs de vértices y enlaces

In [ ]:
val nodes: RDD[(Long, (String, Int))] = sc.parallelize(vertexArray)
val edges: RDD[Edge[Int]] = sc.parallelize(edgeArray)

Crea el grafo

In [ ]:
val friends: Graph[(String, Int), Int] = Graph(nodes, edges)

In [ ]:
friends.edges

In [ ]:
friends.vertices

# Operaciones

Un ejemplo de operación sobre los vértices: filtrado

In [ ]:
val subgroup = friends.vertices.filter {
    case (id, (name,age)) => age > 50 
}

In [ ]:
subgroup.collect.foreach { v => print(s"${v._1} => ${v._2._1}, ${v._2._2}\n") }

# Datos del grafo

In [ ]:
println(friends.numEdges, friends.numVertices)

In [ ]:
friends.inDegrees.collect.foreach( d => print(s"${d._1} -> grado = ${d._2}\n"))

In [ ]:
friends.outDegrees
friends.degrees.collect.foreach( d => print(s"${d._1} -> grado = ${d._2}\n"))

# Tripletas

La clase [EdgeTriplet](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.graphx.EdgeTriplet) agrupa en un objeto un conjunto _nodo -> enlace -> nodo_

<img src='img/triplet.png' width="600">

In [ ]:
friends.triplets.take(1)

Un objeto `triplet` tiene:

` triplet                       
      .srcAttr: (String, Int) 
      .dstAttr: (String, Int) 
      .attr: Int              
      .srcId: VertexId        
      .dstId: VertexId         
 `
 
 Recorremos todas las tripletas

In [ ]:
for (tpl <- friends.triplets.collect) { 
    print( s"${tpl.srcAttr._1} --(${tpl.attr})--> ${tpl.dstAttr._1}\n") 
}

Buscamos ahora las relaciones flojas:

In [ ]:
friends.triplets.filter( _.attr < 3 ).collect.foreach {
    t => print( s"${t.srcAttr._1} --(${t.attr})--> ${t.dstAttr._1}\n")
}

# Procesado

Como ejemplo de operación sobre un grafo, veamos un [recuento de triángulos](https://spark.apache.org/docs/latest/graphx-programming-guide.html#triangle-counting): método [`triangleCount`](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.graphx.lib.TriangleCount$)

In [ ]:
val fp = friends.partitionBy(PartitionStrategy.RandomVertexCut)

In [ ]:
val triCounts = fp.triangleCount().vertices

Para ver los triángulos asignados a cada vértice hacemos un `join` con el RDD de vértices

In [ ]:
val usersTri = friends.vertices.join(triCounts).take(2)

Escribiéndolo más claro:

In [ ]:
friends.vertices.join(triCounts).collect.foreach {
    v => print(s"${v._2._1._1} : ${v._2._2}\n")
}